# SDK Reference Table - `table()` - read

Ocean Data Platform offers both API and Python SDK interfaces. This notebook highlights the Python SDK.

## Installation

```bash
pip install -U odp-sdk
```

## Client Initialization

In [1]:
from odp.client import Client

In [2]:
import pyarrow as pa

In [3]:
# Auto authentication which opens browser to performance authentication process (not in our Workspaces)
client = Client()

In [5]:
# API Key authentication (don't need to open browser).
# You can generate an API key in the Ocean Data Platform web interface, under your user profile.
client = Client(api_key="your-api-key")

## Dataset Access

With an initialized `Client` you can access different datasets by using the datasets' UUID. The easiest way is to use https://app.hubocean.earth/catalog to search for datasets and find the UUID (click API).

For the Table examples we are using a dataset from Brazil provided from one of our partners: 

**Example Dataset**: PGS Biota Data - Mammal and Turtle Observations (Darwin Core Format)  
**Dataset ID**: `1d801817-742b-4867-82cf-5597673524eb`  
**Columns**:`occurrenceID`,`verbatimIdentification`,`scientificName`,`scientificNameID`,`lifeStage`,`individualCount`,`basisOfRecord`,`minimumDepthInMeters`,`eventDate`,`occurrenceRemarks`,`decimalLongitude`,`decimalLatitude`,`footprintWKT`,`license`,`occurrenceStatus`,`geodeticDatum`,`datasetName`,`institutionCode`,`otherCatalogNumbers`

## Get Dataset

In [4]:
# Get dataset
dataset = client.dataset("1d801817-742b-4867-82cf-5597673524eb")

The `dataset` from this UUID will be used in the examples below.

## Get Dataset Schema and Statistics

In [5]:
# Get table schema
schema = dataset.table.schema()  # Returns pyarrow.Schema or None
print(f"Available columns: {schema}")

Available columns: occurrenceID: string
verbatimIdentification: string
scientificName: string
scientificNameID: string
lifeStage: string
individualCount: double
basisOfRecord: string
minimumDepthInMeters: double
eventDate: string
occurrenceRemarks: string
decimalLongitude: double
decimalLatitude: double
footprintWKT: string
  -- field metadata --
  isGeometry: '1'
  class: 'geometry'
  index: '1'
license: string
occurrenceStatus: string
geodeticDatum: string
datasetName: string
institutionCode: string
otherCatalogNumbers: string


In [6]:
# Get table statistics  
stats = dataset.table.stats()
print(f"Total observations: {stats.num_rows:,}")  
print(f"Dataset size: {stats.size:,} bytes")

Total observations: 2,241
Dataset size: 350,374 bytes


## Query Table Data

You query the Table data by using `table.select()` followed by how you want to receive the results.

There are three different ways of receiving the results from the query:

Single batch
- A GeoPandas GeoDataFrame containing all the data (for smaller datasets or a quick view of the data): `dataset.table.select().all().dataframe()`

Streaming batches
- A stream of GeoPandas GeoDataFrames (if the dataset is too large): `dataset.table.select().dataframes()`
- A stream of PyArrow RecordBatches (a more performant way that is recommended): `dataset.table.select().batches()`

### Single batch (for smaller datasets)
A useful way to get the data directly into a single Pandas DataFrame if you are dealing with small datasets.

In [7]:
# Get all marine observations as single pandas DataFrame
result_dataframe = dataset.table.select().all().dataframe()
print(f"Complete dataset: {len(result_dataframe)} marine observations") 

Complete dataset: 2241 marine observations


You can protect from memory overflow by setting:
* max_row (the maximum of rows to be returned in the Python DataFrame
* max_time (time out threshold)

In [8]:
# Get all marine observations as single pandas DataFrame
result_dataframe = dataset.table.select().all(max_rows=10_000_000_000, max_time=30.0).dataframe()
print(f"Complete dataset: {len(result_dataframe)} marine observations") 

Complete dataset: 2241 marine observations


### Streaming batches query
Ocean datasets are usually quite large and often it is better to get the data streaming.

You have two different ways of streaming the: Pandas DataFrames (dataframes), and PyArrow RecordBatch (batches). Working with PyArrow has a performance advantage (memory efficient) and recommended if you are familiar with PyArrow (https://arrow.apache.org/docs/python/index.html), but Pandas is often what most users are more familiar with. However, it easy to convert from PyArrow RecordBatch to Pandas DataFrame.

The streaming is design to allow you stop the streaming at any point when you are done with the operations.

In [9]:
# Iterate by Pandas DataFrames - convenient for analysis
for dataframe_batch in dataset.table.select().dataframes():
    print(f"Analyzing DataFrame batch: {len(dataframe_batch)} observations")
    # Marine biology analysis on chunk
    depth_stats = dataframe_batch['minimumDepthInMeters'].describe()
    print(f"Depth statistics: {depth_stats}")

Analyzing DataFrame batch: 2241 observations
Depth statistics: count    2207.000000
mean     1672.336792
std       997.570882
min         9.500000
25%       858.000000
50%      1738.000000
75%      2322.500000
max      4836.000000
Name: minimumDepthInMeters, dtype: float64


In [10]:
# Iterate by batches (PyArrow RecordBatch) - memory efficient for large datasets
for batch in dataset.table.select().batches():
    print(f"Processing batch with {batch.num_rows} observations")
    # Convert to Pandas
    df_batch = batch.to_pandas()
    # Process marine species in this batch
    unique_species = df_batch['scientificName'].nunique()
    print(f"Found {unique_species} unique species in this batch")

Processing batch with 2241 observations
Found 36 unique species in this batch


### Basic select() Operations

Within the select() method you can pass operators to narrow down:
* Comparison: `AND`, `OR`, `NOT` 
* Logical: `>`, `<`, `>=`, `<=`, `==`, `!=` as well as `IS NULL`,`IS NOT NULL`
* Geospatial: `within`, `intersects`, `contains`

Examples are shown with single batch method, but works in the same way for streaming methods.

In [12]:
# Select specific
dataframe = dataset.table.select("scientificName == 'Balaenoptera'").all().dataframe()
print(f"Number of rows: {dataframe.shape[0]}")

Number of rows: 143


In [13]:
# Explicit parameter
dataframe = dataset.table.select(filter="scientificName == 'Balaenoptera'").all().dataframe()
print(f"Number of rows: {dataframe.shape[0]}")

Number of rows: 143


In [14]:
# Select specific columns which is more efficient than selecting all columns and filtering in Python
dataframe = dataset.table.select(
    "minimumDepthInMeters > 100", 
    cols=["scientificName", "lifeStage", "minimumDepthInMeters", "eventDate"]
).all().dataframe()
print(f"Number of rows: {dataframe.shape[0]}")

Number of rows: 2170


In [16]:
# Select with multiple variables
dataframe = dataset.select(
    "scientificName == 'Balaenoptera' AND minimumDepthInMeters > 100"
).all().dataframe()
print(f"Number of rows: {dataframe.shape[0]}")

Number of rows: 141


In [17]:
# Select with named bind variables for safe, efficient queries:
dataframe = dataset.table.select(
    "scientificName == $species",
    vars={
        "species": "Balaenoptera"
    }
).all().dataframe()
print(f"Number of rows: {dataframe.shape[0]}")

Number of rows: 143


In [18]:
# How to work with geo
dataframe = dataset.table.select(
    'footprintWKT within $area', 
    vars={"area": "POLYGON((-37 -12, -45 -26, -40 -28, -33 -13, -37 -12))"},
).all().dataframe()
print(f"Number of rows: {dataframe.shape[0]}")

Number of rows: 1395


### Aggregations
Some description on aggregations
- max
- min
- sum
- count
- mean (average)

Geo aggregations
- h3

In [19]:
# Aggregate by a column
dataframe = dataset.table.aggregate(
    group_by="lifeStage",
    aggr={"minimumDepthInMeters": "mean"}
)
print(dataframe)

             *  minimumDepthInMeters
                                    
None       498           2108.367470
adult     1606           1544.799809
juvenile   137           1550.766423


In [20]:
# Aggregate by a column combined with a query
dataframe = dataset.table.aggregate(
    group_by="scientificName",
    filter="scientificName IS NOT NULL AND minimumDepthInMeters IS NOT NULL",
    aggr={
        "minimumDepthInMeters": "mean"
    }
)
print(dataframe.iloc[0:5])

                              *  minimumDepthInMeters
                                                     
Balaenoptera                143           1796.202797
Balaenoptera acutorostrata   51           1925.647059
Balaenoptera bonaerensis      4           2616.000000
Balaenoptera brydei           1           1456.000000
Balaenoptera edeni            4           2027.000000


In [21]:
# Aggregate without grouping
dataframe = dataset.table.aggregate(
    group_by='"TOTAL"',  # Special value
    aggr={ 
        "minimumDepthInMeters": "max"
    }
)
print(dataframe)

          *  minimumDepthInMeters
                                 
TOTAL  2241                4836.0


In [23]:
# Aggregate by h3 hexagons
dataframe = dataset.table.aggregate(
    group_by="h3(footprintWKT, 5)", # Arguments: Column containing the geometry, and resolution between 0 and 15 https://h3geo.org/docs/core-library/restable/
    filter="footprintWKT IS NOT NULL",
    aggr={
        "minimumDepthInMeters": "mean"
    }
)
print(dataframe.iloc[0:])

                 *  minimumDepthInMeters
                                        
85801047fffffff  1                   9.5
85801203fffffff  3                   NaN
85801207fffffff  2                   NaN
8580120bfffffff  1                   NaN
8580120ffffffff  1                   NaN
...             ..                   ...
85c51c63fffffff  1                 458.0
85c51d2bfffffff  1                4000.0
85c51d47fffffff  5                4000.0
85c51d57fffffff  1                4000.0
85c51d73fffffff  4                4000.0

[582 rows x 2 columns]


### Performance tips



1. Use column selection: Only select columns you need.
2. Use bind variables: More efficient than string concatenation (safer as well).
3. Filter early: Apply filters in the query rather than in Python.
4. Consider aggregation: Use aggregate() instead of selecting all data and aggregating in Python.
5. Use Streaming: Handle large datasets by streaming and iterate them.  

### Error handling 

In [33]:
# 
try:
    result = dataset.table.select("invalid_column =! 5").all().dataframe()
except ValueError as e:
    print(f"Query error: {e}")

Query error: {"error": "Can't parse:\ninvalid_column =! 5\n                ^\n\nNo terminal matches '!' in the current parser context, at line 1 col 17\n\ninvalid_column =! 5\n                ^\nExpected one of: \n\t* NAME\n\t* LPAR\n\t* /(True|False|None|null|true|false)/\n\t* ESCAPED_STRING\n\t* SIGNED_NUMBER\n\t* DOLLAR\n\t* TILDE\n\t* /'[^']*'/\n\t* QMARK\n\nPrevious tokens: Token('__ANON_1', '=')\n", "request-id": "d8d782dcaab2"}
